In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchaudio
from load_data import *
from transformers import Wav2Vec2Processor
from transformers import CamembertTokenizer

raw_data = load_all_ipus("Dataset/transcr")
filepath = "Dataset/audio/2_channels/"

wave2vec_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(wave2vec_name)

bert_name = 'camembert-base'
tokenizer = CamembertTokenizer.from_pretrained(bert_name)

def get_audio(i, raw_data, filepath):
    audio_file_path =  filepath + raw_data["dyad"][i].replace("transcr\\","") + ".wav"
    audio_tensor, sampling_rate = torchaudio.load(audio_file_path)
    audio_tensor = processor(audio_tensor, return_tensors="pt", sampling_rate = sampling_rate).input_values.squeeze(0)
    stop = int(sampling_rate*raw_data["stop"][i])
    start = stop-sampling_rate*1
    if start < 0:
        sample_tensor = audio_tensor[:,0:stop]
        sample_tensor = torch.cat((torch.zeros((2,abs(start))),sample_tensor),dim=1)
    else:
        sample_tensor = audio_tensor[:,start:stop]
    return sample_tensor

def get_text(i, raw_data):
    text = raw_data["text"][i]
    text_tokenized = tokenizer(text, return_tensors="pt")['input_ids']
    text_tokenized = torch.cat((torch.tensor([[1]*20]),text_tokenized),dim=1)
    text_tokenized = text_tokenized[:,-20:]
    return text_tokenized.squeeze(0)

def get_label(i, raw_data):
    return raw_data["turn_after"].astype("float32")[i]

class DataGenerator(Dataset):
    
    def __init__(self, raw_data, filepath):
        self.raw_data = raw_data
        self.filepath = filepath
    
    def __getitem__(self, i):
        return {"audio" : get_audio(i, self.raw_data, self.filepath),
                "text" : get_text(i, self.raw_data),
                "label" : get_label(i, self.raw_data)}
        
    def __len__(self):
        return len(self.raw_data)
    
def create_dataloader(generator):
    
    dataloader = DataLoader(generator,
                            batch_size=64,
                            shuffle=True,
                            drop_last=True,)
    
    return dataloader

(16400, 12)
is_main_speaker      0.739207
turn_at_start        0.184756
turn_after           0.184756
turn_start_word     10.250372
yield_at_end         0.188415
request_at_start     0.198537
dtype: float64


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
import torch
from tqdm import tqdm
from colorama import Fore, Style
from data_processing import *
from transformers import Wav2Vec2Model
from transformers import CamembertModel
from icecream import ic

########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wave2vec_name = "facebook/wav2vec2-base-960h"
wave2vec_model = Wav2Vec2Model.from_pretrained(wave2vec_name)
wave2vec_model = wave2vec_model.to(device)

bert_name = 'camembert-base'
bert_model = CamembertModel.from_pretrained(bert_name)
bert_model = bert_model.to(device)

for param in wave2vec_model.parameters():
    param.requires_grad = False

for param in bert_model.parameters():
    param.requires_grad = False
    
########################################################################

class Model(torch.nn.Module):
    """
    Model using covolutional neural net architecture.
    """
    def __init__(self, input_dim, hidden_dim, output_dim, device):
        
        super(Model, self).__init__()

        self.device = device
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        # Define model components
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim)
        self.dropout = torch.nn.Dropout(p=0.1)
        self.linear2 = torch.nn.Linear(hidden_dim, output_dim)
        
###################
        
    def parameters_number(self):
        return(sum(p.numel() for p in self.parameters() if p.requires_grad))
        
    def forward(self, input_data):
        # Define the forward pass using the model components
        
        input_data['audio'] = input_data['audio'].to(self.device)
        input_data['text'] = input_data['text'].to(self.device)
        input_data['label'] = input_data['label'].to(self.device)

        channel0 = input_data['audio'][:,0,:]
        channel1 = input_data['audio'][:,1,:]
        
        wave2vec_output0 = wave2vec_model(channel0)
        wave2vec_output1 = wave2vec_model(channel1)
        
        wave2vec_output0 = wave2vec_output0.last_hidden_state
        wave2vec_output1 = wave2vec_output1.last_hidden_state
        
        wave2vec_output0 = torch.nn.functional.max_pool1d(wave2vec_output0, kernel_size=6)
        wave2vec_output1 = torch.nn.functional.max_pool1d(wave2vec_output1, kernel_size=6)
        
        bert_output = bert_model(input_data['text'])
        bert_output = bert_output.last_hidden_state
        
        wave2vec_output0 = torch.flatten(wave2vec_output0, start_dim=1)
        wave2vec_output1 = torch.flatten(wave2vec_output1, start_dim=1)
        bert_output = torch.flatten(bert_output, start_dim=1)

        # Concatenate or combine the outputs as needed
        combined_output = torch.cat((wave2vec_output0, wave2vec_output1, bert_output), dim=1)

        # Apply linear layers
        linear1_output = torch.relu(self.linear1(combined_output))
        final_output = self.linear2(self.dropout(linear1_output))

        return torch.softmax(final_output, dim=1)
    
    def evaluate(self, dataloader):
        self.eval()

        # Define the loss function
        loss_function = torch.nn.CrossEntropyLoss()

        total_loss = 0.0
        true_positive_1 = torch.tensor(0).to(self.device)
        false_positive_1 = torch.tensor(0).to(self.device)
        false_negative_1 = torch.tensor(0).to(self.device)
        
        true_positive_0 = torch.tensor(0).to(self.device)
        false_positive_0 = torch.tensor(0).to(self.device)
        false_negative_0 = torch.tensor(0).to(self.device)

        with torch.no_grad():  # Disable gradient computation during validation
            for _, batch in enumerate(tqdm(dataloader, desc="Validating")):
                
                output = self.forward(batch)
                labels = batch["label"].long()

                loss = loss_function(output, labels)
                total_loss += loss.item()

                _, predicted = torch.max(output, 1)
                
                true_positive_1 += torch.sum((predicted == labels) * (labels == 1))
                false_positive_1 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 1))
                false_negative_1 += torch.sum(((1-predicted) == labels) * (labels == 1))
                
                true_positive_0 += torch.sum((predicted == labels) * (labels == 0))
                false_positive_0 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 0))
                false_negative_0 += torch.sum(((1-predicted) == labels) * (labels == 0))
                
        precision_1 = true_positive_1 / max((true_positive_1 + false_positive_1), 1)
        recall_1 = true_positive_1 / max((true_positive_1 + false_negative_1), 1) 
        
        precision_0 = true_positive_0 / max((true_positive_0 + false_positive_0), 1)
        recall_0 = true_positive_0 / max((true_positive_0 + false_negative_0), 1) 
        
        f1_1 = 2 * (precision_1 * recall_1) / max((precision_1 + recall_1), 1)
        f1_0 = 2 * (precision_0 * recall_0) / max((precision_0 + recall_0), 1)
        
        print(f"Classe {Fore.RED}0{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_0 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_0 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_0 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Classe {Fore.RED}1{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_1 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_1 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_1 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Score : {(f1_0*0.18 + f1_1*(1-0.18))}")
                        
    def train_one_epoch(self, dataloader):
        
        self.train(True)
        
        optimizer = torch.optim.Adam(self.parameters())
        # loss_function = torch.nn.CrossEntropyLoss()
        
        try:
            for _, batch in enumerate(tqdm(dataloader, desc="Training")):
                
                try:
                
                    optimizer.zero_grad()
                    
                    output = self.forward(batch)
                    labels = batch["label"].long()
                    _, predicted = torch.max(output, 1)
                    
                    true_positive_0 = (output * (1 - labels).unsqueeze(1))[:, 0].sum()
                    false_positive_0 = ((1 - output) * labels.unsqueeze(1))[:, 0].sum()
                    false_negative_0 = ((1 - output) * (1 - labels).unsqueeze(1))[:, 0].sum()

                    true_positive_1 = (output * labels.unsqueeze(1))[:, 1].sum()
                    false_positive_1 = (output * (1 - labels).unsqueeze(1))[:, 1].sum()
                    false_negative_1 = ((1 - output) * labels.unsqueeze(1))[:, 1].sum()

                    epsilon = torch.tensor([1e-7]).to(self.device) # 1e-7
                    
                    precision_0 = true_positive_0 / (true_positive_0 + false_positive_0 + epsilon)
                    recall_0 = true_positive_0 / (true_positive_0 + false_negative_0 + epsilon)
                    f1_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0 + epsilon)

                    precision_1 = true_positive_1 / (true_positive_1 + false_positive_1 + epsilon)
                    recall_1 = true_positive_1 / (true_positive_1 + false_negative_1 + epsilon)
                    f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1 + epsilon)
                
                    loss = -torch.log((f1_0*0.18 + f1_1*(1-0.18)) + epsilon)
                    
                    # ic(output, labels)
                    # ic(true_positive_0, false_positive_0, false_negative_0, true_positive_1, false_positive_1, false_negative_1)
                    # ic(precision_0, recall_0, f1_0, precision_1, recall_1, f1_1)
                    # print("Successfully trained a batch")
                    
                    ic(loss)
                    loss.backward()
                    
                    # labels = batch["label"].long()
                    # loss = loss_function(output, labels)
                    # loss.backward()
                    
                    optimizer.step()
                    
                except StopIteration as e:
                    print(f"An error occurred: {e}")   
                    # Catch StopIteration and continue to the next batch
                    pass
        except Exception as e:
            # Handle other exceptions if needed
            print(f"An error occurred: {e}")      

    def train_loop(self, generator, nb_epoch):
        
        self.to(self.device)

        data = generator.raw_data

        train_data = data.sample(frac=0.9,random_state=200)
        test_data = data.drop(train_data.index)

        test_data.reset_index(drop=True, inplace=True)
        train_data.reset_index(drop=True, inplace=True)

        test_generator = DataGenerator(test_data, filepath)
        test_loader = create_dataloader(test_generator)
        
        for epoch_number in range(nb_epoch):

            train_subdata = train_data.sample(frac=0.9,random_state=200)
            val_subdata = train_data.drop(train_subdata.index)
            
            train_subdata.reset_index(drop=True, inplace=True)
            val_subdata.reset_index(drop=True, inplace=True)
            
            train_subgenerator = DataGenerator(train_subdata, filepath)
            val_subgenerator = DataGenerator(val_subdata, filepath)

            train_loader = create_dataloader(train_subgenerator)
            val_loader = create_dataloader(val_subgenerator)

            print("")    
            print(f'{Fore.GREEN}EPOCH {epoch_number + 1}:{Style.RESET_ALL}')
            
            # Train for one epoch
            self.train_one_epoch(train_loader)

            # Validate on the validation subset
            print(f'{Fore.CYAN}Validation :{Style.RESET_ALL}')
            self.evaluate(val_loader)
            print(f'{Fore.YELLOW}Test :{Style.RESET_ALL}')
            self.evaluate(test_loader)

########################################################################

generator = DataGenerator(raw_data.iloc[:2048], filepath)
model = Model(27904, 16, 2, device)

# dataloader = create_dataloader(generator)
# model.evaluate(dataloader)

print("Nombre de paramètres du model:", model.parameters_number())

model.train_loop(generator, 5)

(16400, 12)
is_main_speaker      0.739207
turn_at_start        0.184756
turn_after           0.184756
turn_start_word     10.250372
yield_at_end         0.188415
request_at_start     0.198537
dtype: float64


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Nombre de paramètres du model: 446514

EPOCH 1:


Training:   0%|          | 0/25 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
ic| loss: tensor([1.0399], device='cuda:0', grad_fn=<NegBackward0>)
Training: 100%|██████████| 25/25 [11:36<00:00, 27.85s/it]


Validation :


...

Validation :


Validating: 100%|██████████| 2/2 [01:00<00:00, 30.19s/it]


Classe 0 | Precision: 84.69%, Recall: 79.05%, F1 Score: 81.77%
Classe 1 | Precision: 26.67%, Recall: 34.78%, F1 Score: 18.55%
Score : 0.29930806159973145
Test :


Validating: 100%|██████████| 3/3 [01:20<00:00, 26.97s/it]

Classe 0 | Precision: 84.89%, Recall: 75.64%, F1 Score: 80.00%
Classe 1 | Precision: 28.30%, Recall: 41.67%, F1 Score: 23.58%
Score : 0.33739620447158813


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchaudio
from load_data import *
from transformers import Wav2Vec2Processor
from transformers import CamembertTokenizer

raw_data = load_all_ipus("Dataset/transcr")
filepath = "Dataset/audio/2_channels/"

wave2vec_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(wave2vec_name)

bert_name = 'camembert-base'
tokenizer = CamembertTokenizer.from_pretrained(bert_name)

def get_audio(i, raw_data, filepath):
    audio_file_path =  filepath + raw_data["dyad"][i].replace("transcr\\","") + ".wav"
    audio_tensor, sampling_rate = torchaudio.load(audio_file_path)
    audio_tensor = processor(audio_tensor, return_tensors="pt", sampling_rate = sampling_rate).input_values.squeeze(0)
    stop = int(sampling_rate*raw_data["stop"][i])
    start = stop-sampling_rate*1
    if start < 0:
        sample_tensor = audio_tensor[:,0:stop]
        sample_tensor = torch.cat((torch.zeros((2,abs(start))),sample_tensor),dim=1)
    else:
        sample_tensor = audio_tensor[:,start:stop]
    return sample_tensor

def get_text(i, raw_data):
    text = raw_data["text"][i]
    text_tokenized = tokenizer(text, return_tensors="pt")['input_ids']
    text_tokenized = torch.cat((torch.tensor([[1]*20]),text_tokenized),dim=1)
    text_tokenized = text_tokenized[:,-20:]
    return text_tokenized.squeeze(0)

def get_label(i, raw_data):
    return raw_data["turn_after"].astype("float32")[i]

class DataGenerator(Dataset):
    
    def __init__(self, raw_data, filepath):
        self.raw_data = raw_data
        self.filepath = filepath
    
    def __getitem__(self, i):
        return {"audio" : get_audio(i, self.raw_data, self.filepath),
                "text" : get_text(i, self.raw_data),
                "label" : get_label(i, self.raw_data)}
        
    def __len__(self):
        return len(self.raw_data)
    
def create_dataloader(generator):
    
    dataloader = DataLoader(generator,
                            batch_size=64,
                            shuffle=True,
                            drop_last=True,)
    
    return dataloader

(16400, 12)
is_main_speaker      0.739207
turn_at_start        0.184756
turn_after           0.184756
turn_start_word     10.250372
yield_at_end         0.188415
request_at_start     0.198537
dtype: float64


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchaudio
from load_data import *
from transformers import Wav2Vec2Processor
from transformers import CamembertTokenizer

raw_data = load_all_ipus("Dataset/transcr")
filepath = "Dataset/audio/2_channels/"

wave2vec_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(wave2vec_name)

bert_name = 'camembert-base'
tokenizer = CamembertTokenizer.from_pretrained(bert_name)

def get_audio(i, raw_data, filepath):
    audio_file_path =  filepath + raw_data["dyad"][i].replace("transcr\\","") + ".wav"
    audio_tensor, sampling_rate = torchaudio.load(audio_file_path)
    audio_tensor = processor(audio_tensor, return_tensors="pt", sampling_rate = sampling_rate).input_values.squeeze(0)
    stop = int(sampling_rate*raw_data["stop"][i])
    start = stop-sampling_rate*1
    if start < 0:
        sample_tensor = audio_tensor[:,0:stop]
        sample_tensor = torch.cat((torch.zeros((2,abs(start))),sample_tensor),dim=1)
    else:
        sample_tensor = audio_tensor[:,start:stop]
    return sample_tensor

def get_text(i, raw_data):
    text = raw_data["text"][i]
    text_tokenized = tokenizer(text, return_tensors="pt")['input_ids']
    text_tokenized = torch.cat((torch.tensor([[1]*20]),text_tokenized),dim=1)
    text_tokenized = text_tokenized[:,-20:]
    return text_tokenized.squeeze(0)

def get_label(i, raw_data):
    return raw_data["turn_after"].astype("float32")[i]

class DataGenerator(Dataset):
    
    def __init__(self, raw_data, filepath):
        self.raw_data = raw_data
        self.filepath = filepath
    
    def __getitem__(self, i):
        return {"audio" : get_audio(i, self.raw_data, self.filepath),
                "text" : get_text(i, self.raw_data),
                "label" : get_label(i, self.raw_data)}
        
    def __len__(self):
        return len(self.raw_data)
    
def create_dataloader(generator):
    
    dataloader = DataLoader(generator,
                            batch_size=64,
                            shuffle=True,
                            drop_last=True,)
    
    return dataloader

(16400, 12)
is_main_speaker      0.739207
turn_at_start        0.184756
turn_after           0.184756
turn_start_word     10.250372
yield_at_end         0.188415
request_at_start     0.198537
dtype: float64


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
generator = DataGenerator(raw_data.iloc[2048:2048*3], filepath)

In [6]:
model.train_loop(generator, 5)


EPOCH 1:


Training:  10%|▉         | 5/51 [02:36<23:58, 31.28s/it]


An error occurred: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).
Validation :


Validating: 100%|██████████| 5/5 [02:41<00:00, 32.21s/it]


Classe 0 | Precision: 84.49%, Recall: 80.23%, F1 Score: 82.31%
Classe 1 | Precision: 32.00%, Recall: 38.71%, F1 Score: 24.77%
Score : 0.3512994647026062
Test :


...

An error occurred: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).
Validation :


Validating: 100%|██████████| 5/5 [02:00<00:00, 24.07s/it]


Classe 0 | Precision: 84.72%, Recall: 74.62%, F1 Score: 79.35%
Classe 1 | Precision: 27.47%, Recall: 41.67%, F1 Score: 22.89%
Score : 0.33055102825164795
Test :


Validating:   0%|          | 0/6 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [7]:
torch.save(model, "checkpoint1.pth")

In [8]:
import torch
from tqdm import tqdm
from colorama import Fore, Style
from data_processing import *
from transformers import Wav2Vec2Model
from transformers import CamembertModel
from icecream import ic

########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wave2vec_name = "facebook/wav2vec2-base-960h"
wave2vec_model = Wav2Vec2Model.from_pretrained(wave2vec_name)
wave2vec_model = wave2vec_model.to(device)

bert_name = 'camembert-base'
bert_model = CamembertModel.from_pretrained(bert_name)
bert_model = bert_model.to(device)

for param in wave2vec_model.parameters():
    param.requires_grad = False

for param in bert_model.parameters():
    param.requires_grad = False
    
########################################################################

class Model(torch.nn.Module):
    """
    Model using covolutional neural net architecture.
    """
    def __init__(self, input_dim, hidden_dim, output_dim, device):
        
        super(Model, self).__init__()

        self.device = device
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        # Define model components
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim)
        self.dropout = torch.nn.Dropout(p=0.1)
        self.linear2 = torch.nn.Linear(hidden_dim, output_dim)
        
###################
        
    def parameters_number(self):
        return(sum(p.numel() for p in self.parameters() if p.requires_grad))
        
    def forward(self, input_data):
        # Define the forward pass using the model components
        
        input_data['audio'] = input_data['audio'].to(self.device)
        input_data['text'] = input_data['text'].to(self.device)
        input_data['label'] = input_data['label'].to(self.device)

        channel0 = input_data['audio'][:,0,:]
        channel1 = input_data['audio'][:,1,:]
        
        wave2vec_output0 = wave2vec_model(channel0)
        wave2vec_output1 = wave2vec_model(channel1)
        
        wave2vec_output0 = wave2vec_output0.last_hidden_state
        wave2vec_output1 = wave2vec_output1.last_hidden_state
        
        wave2vec_output0 = torch.nn.functional.max_pool1d(wave2vec_output0, kernel_size=6)
        wave2vec_output1 = torch.nn.functional.max_pool1d(wave2vec_output1, kernel_size=6)
        
        bert_output = bert_model(input_data['text'])
        bert_output = bert_output.last_hidden_state
        
        wave2vec_output0 = torch.flatten(wave2vec_output0, start_dim=1)
        wave2vec_output1 = torch.flatten(wave2vec_output1, start_dim=1)
        bert_output = torch.flatten(bert_output, start_dim=1)

        # Concatenate or combine the outputs as needed
        combined_output = torch.cat((wave2vec_output0, wave2vec_output1, bert_output), dim=1)

        # Apply linear layers
        linear1_output = torch.relu(self.linear1(combined_output))
        final_output = self.linear2(self.dropout(linear1_output))

        return torch.softmax(final_output, dim=1)
    
    def evaluate(self, dataloader):
        self.eval()

        # Define the loss function
        loss_function = torch.nn.CrossEntropyLoss()

        total_loss = 0.0
        true_positive_1 = torch.tensor(0).to(self.device)
        false_positive_1 = torch.tensor(0).to(self.device)
        false_negative_1 = torch.tensor(0).to(self.device)
        
        true_positive_0 = torch.tensor(0).to(self.device)
        false_positive_0 = torch.tensor(0).to(self.device)
        false_negative_0 = torch.tensor(0).to(self.device)

        with torch.no_grad():  # Disable gradient computation during validation
            for _, batch in enumerate(tqdm(dataloader, desc="Validating")):
                
                output = self.forward(batch)
                labels = batch["label"].long()

                loss = loss_function(output, labels)
                total_loss += loss.item()

                _, predicted = torch.max(output, 1)
                
                true_positive_1 += torch.sum((predicted == labels) * (labels == 1))
                false_positive_1 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 1))
                false_negative_1 += torch.sum(((1-predicted) == labels) * (labels == 1))
                
                true_positive_0 += torch.sum((predicted == labels) * (labels == 0))
                false_positive_0 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 0))
                false_negative_0 += torch.sum(((1-predicted) == labels) * (labels == 0))
                
        precision_1 = true_positive_1 / max((true_positive_1 + false_positive_1), 1)
        recall_1 = true_positive_1 / max((true_positive_1 + false_negative_1), 1) 
        
        precision_0 = true_positive_0 / max((true_positive_0 + false_positive_0), 1)
        recall_0 = true_positive_0 / max((true_positive_0 + false_negative_0), 1) 
        
        f1_1 = 2 * (precision_1 * recall_1) / max((precision_1 + recall_1), 1)
        f1_0 = 2 * (precision_0 * recall_0) / max((precision_0 + recall_0), 1)
        
        print(f"Classe {Fore.RED}0{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_0 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_0 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_0 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Classe {Fore.RED}1{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_1 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_1 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_1 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Score : {(f1_0*0.18 + f1_1*(1-0.18))}")
                        
    def train_one_epoch(self, dataloader):
        
        self.train(True)
        
        optimizer = torch.optim.Adam(self.parameters())
        # loss_function = torch.nn.CrossEntropyLoss()
        
        try:
            for _, batch in enumerate(tqdm(dataloader, desc="Training")):
                
                try:
                
                    optimizer.zero_grad()
                    
                    output = self.forward(batch)
                    labels = batch["label"].long()
                    _, predicted = torch.max(output, 1)
                    
                    true_positive_0 = (output * (1 - labels).unsqueeze(1))[:, 0].sum()
                    false_positive_0 = ((1 - output) * labels.unsqueeze(1))[:, 0].sum()
                    false_negative_0 = ((1 - output) * (1 - labels).unsqueeze(1))[:, 0].sum()

                    true_positive_1 = (output * labels.unsqueeze(1))[:, 1].sum()
                    false_positive_1 = (output * (1 - labels).unsqueeze(1))[:, 1].sum()
                    false_negative_1 = ((1 - output) * labels.unsqueeze(1))[:, 1].sum()

                    epsilon = torch.tensor([1e-7]).to(self.device) # 1e-7
                    
                    precision_0 = true_positive_0 / (true_positive_0 + false_positive_0 + epsilon)
                    recall_0 = true_positive_0 / (true_positive_0 + false_negative_0 + epsilon)
                    f1_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0 + epsilon)

                    precision_1 = true_positive_1 / (true_positive_1 + false_positive_1 + epsilon)
                    recall_1 = true_positive_1 / (true_positive_1 + false_negative_1 + epsilon)
                    f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1 + epsilon)
                
                    loss = -torch.log((f1_0*0.18 + f1_1*(1-0.18)) + epsilon)
                    
                    # ic(output, labels)
                    # ic(true_positive_0, false_positive_0, false_negative_0, true_positive_1, false_positive_1, false_negative_1)
                    # ic(precision_0, recall_0, f1_0, precision_1, recall_1, f1_1)
                    # print("Successfully trained a batch")
                    
                    ic(loss)
                    loss.backward()
                    
                    # labels = batch["label"].long()
                    # loss = loss_function(output, labels)
                    # loss.backward()
                    
                    optimizer.step()
                    
                except StopIteration as e:
                    print(f"An error occurred: {e}")   
                    # Catch StopIteration and continue to the next batch
                    pass
        except Exception as e:
            # Handle other exceptions if needed
            print(f"An error occurred: {e}")      

    def train_loop(self, generator, nb_epoch):
        
        self.to(self.device)

        data = generator.raw_data

        train_data = data.sample(frac=0.9,random_state=200)
        test_data = data.drop(train_data.index)

        test_data.reset_index(drop=True, inplace=True)
        train_data.reset_index(drop=True, inplace=True)

        test_generator = DataGenerator(test_data, filepath)
        test_loader = create_dataloader(test_generator)
        
        for epoch_number in range(nb_epoch):

            train_subdata = train_data.sample(frac=0.9,random_state=200)
            val_subdata = train_data.drop(train_subdata.index)
            
            train_subdata.reset_index(drop=True, inplace=True)
            val_subdata.reset_index(drop=True, inplace=True)
            
            train_subgenerator = DataGenerator(train_subdata, filepath)
            val_subgenerator = DataGenerator(val_subdata, filepath)

            train_loader = create_dataloader(train_subgenerator)
            val_loader = create_dataloader(val_subgenerator)

            print("")    
            print(f'{Fore.GREEN}EPOCH {epoch_number + 1}:{Style.RESET_ALL}')
            
            # Train for one epoch
            self.train_one_epoch(train_loader)

            # Validate on the validation subset
            print(f'{Fore.CYAN}Validation :{Style.RESET_ALL}')
            self.evaluate(val_loader)
            print(f'{Fore.YELLOW}Test :{Style.RESET_ALL}')
            self.evaluate(test_loader)

########################################################################

generator = DataGenerator(raw_data.iloc[2048:2048*3], filepath)
model = Model(27904, 16, 2, device)

# dataloader = create_dataloader(generator)
# model.evaluate(dataloader)

print("Nombre de paramètres du model:", model.parameters_number())

model.train_loop(generator, 5)

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Nombre de paramètres du model: 446514

EPOCH 1:


Training:   0%|          | 0/51 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [9]:
import torch
from tqdm import tqdm
from colorama import Fore, Style
from data_processing import *
from transformers import Wav2Vec2Model
from transformers import CamembertModel
from icecream import ic

########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wave2vec_name = "facebook/wav2vec2-base-960h"
wave2vec_model = Wav2Vec2Model.from_pretrained(wave2vec_name)
wave2vec_model = wave2vec_model.to(device)

bert_name = 'camembert-base'
bert_model = CamembertModel.from_pretrained(bert_name)
bert_model = bert_model.to(device)

for param in wave2vec_model.parameters():
    param.requires_grad = False

for param in bert_model.parameters():
    param.requires_grad = False
    
########################################################################

class Model(torch.nn.Module):
    """
    Model using covolutional neural net architecture.
    """
    def __init__(self, input_dim, hidden_dim, output_dim, device):
        
        super(Model, self).__init__()

        self.device = device
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        # Define model components
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim)
        self.dropout = torch.nn.Dropout(p=0.1)
        self.linear2 = torch.nn.Linear(hidden_dim, output_dim)
        
###################
        
    def parameters_number(self):
        return(sum(p.numel() for p in self.parameters() if p.requires_grad))
        
    def forward(self, input_data):
        # Define the forward pass using the model components
        
        input_data['audio'] = input_data['audio'].to(self.device)
        input_data['text'] = input_data['text'].to(self.device)
        input_data['label'] = input_data['label'].to(self.device)

        channel0 = input_data['audio'][:,0,:]
        channel1 = input_data['audio'][:,1,:]
        
        wave2vec_output0 = wave2vec_model(channel0)
        wave2vec_output1 = wave2vec_model(channel1)
        
        wave2vec_output0 = wave2vec_output0.last_hidden_state
        wave2vec_output1 = wave2vec_output1.last_hidden_state
        
        wave2vec_output0 = torch.nn.functional.max_pool1d(wave2vec_output0, kernel_size=6)
        wave2vec_output1 = torch.nn.functional.max_pool1d(wave2vec_output1, kernel_size=6)
        
        bert_output = bert_model(input_data['text'])
        bert_output = bert_output.last_hidden_state
        
        wave2vec_output0 = torch.flatten(wave2vec_output0, start_dim=1)
        wave2vec_output1 = torch.flatten(wave2vec_output1, start_dim=1)
        bert_output = torch.flatten(bert_output, start_dim=1)

        # Concatenate or combine the outputs as needed
        combined_output = torch.cat((wave2vec_output0, wave2vec_output1, bert_output), dim=1)

        # Apply linear layers
        linear1_output = torch.relu(self.linear1(combined_output))
        final_output = self.linear2(self.dropout(linear1_output))

        return torch.softmax(final_output, dim=1)
    
    def evaluate(self, dataloader):
        self.eval()

        # Define the loss function
        loss_function = torch.nn.CrossEntropyLoss()

        total_loss = 0.0
        true_positive_1 = torch.tensor(0).to(self.device)
        false_positive_1 = torch.tensor(0).to(self.device)
        false_negative_1 = torch.tensor(0).to(self.device)
        
        true_positive_0 = torch.tensor(0).to(self.device)
        false_positive_0 = torch.tensor(0).to(self.device)
        false_negative_0 = torch.tensor(0).to(self.device)

        with torch.no_grad():  # Disable gradient computation during validation
            for _, batch in enumerate(tqdm(dataloader, desc="Validating")):
                
                output = self.forward(batch)
                labels = batch["label"].long()

                loss = loss_function(output, labels)
                total_loss += loss.item()

                _, predicted = torch.max(output, 1)
                
                true_positive_1 += torch.sum((predicted == labels) * (labels == 1))
                false_positive_1 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 1))
                false_negative_1 += torch.sum(((1-predicted) == labels) * (labels == 1))
                
                true_positive_0 += torch.sum((predicted == labels) * (labels == 0))
                false_positive_0 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 0))
                false_negative_0 += torch.sum(((1-predicted) == labels) * (labels == 0))
                
        precision_1 = true_positive_1 / max((true_positive_1 + false_positive_1), 1)
        recall_1 = true_positive_1 / max((true_positive_1 + false_negative_1), 1) 
        
        precision_0 = true_positive_0 / max((true_positive_0 + false_positive_0), 1)
        recall_0 = true_positive_0 / max((true_positive_0 + false_negative_0), 1) 
        
        f1_1 = 2 * (precision_1 * recall_1) / max((precision_1 + recall_1), 1)
        f1_0 = 2 * (precision_0 * recall_0) / max((precision_0 + recall_0), 1)
        
        print(f"Classe {Fore.RED}0{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_0 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_0 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_0 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Classe {Fore.RED}1{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_1 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_1 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_1 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Score : {(f1_0*0.18 + f1_1*(1-0.18))}")
                        
    def train_one_epoch(self, dataloader):
        
        self.train(True)
        
        optimizer = torch.optim.Adam(self.parameters())
        # loss_function = torch.nn.CrossEntropyLoss()
        
        try:
            for _, batch in enumerate(tqdm(dataloader, desc="Training")):
                
                try:
                
                    optimizer.zero_grad()
                    
                    output = self.forward(batch)
                    labels = batch["label"].long()
                    _, predicted = torch.max(output, 1)
                    
                    true_positive_0 = (output * (1 - labels).unsqueeze(1))[:, 0].sum()
                    false_positive_0 = ((1 - output) * labels.unsqueeze(1))[:, 0].sum()
                    false_negative_0 = ((1 - output) * (1 - labels).unsqueeze(1))[:, 0].sum()

                    true_positive_1 = (output * labels.unsqueeze(1))[:, 1].sum()
                    false_positive_1 = (output * (1 - labels).unsqueeze(1))[:, 1].sum()
                    false_negative_1 = ((1 - output) * labels.unsqueeze(1))[:, 1].sum()

                    epsilon = torch.tensor([1e-7]).to(self.device) # 1e-7
                    
                    precision_0 = true_positive_0 / (true_positive_0 + false_positive_0 + epsilon)
                    recall_0 = true_positive_0 / (true_positive_0 + false_negative_0 + epsilon)
                    f1_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0 + epsilon)

                    precision_1 = true_positive_1 / (true_positive_1 + false_positive_1 + epsilon)
                    recall_1 = true_positive_1 / (true_positive_1 + false_negative_1 + epsilon)
                    f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1 + epsilon)
                
                    loss = -torch.log((f1_0*0.18 + f1_1*(1-0.18)) + epsilon)
                    
                    # ic(output, labels)
                    # ic(true_positive_0, false_positive_0, false_negative_0, true_positive_1, false_positive_1, false_negative_1)
                    # ic(precision_0, recall_0, f1_0, precision_1, recall_1, f1_1)
                    # print("Successfully trained a batch")
                    
                    ic(loss)
                    loss.backward()
                    
                    # labels = batch["label"].long()
                    # loss = loss_function(output, labels)
                    # loss.backward()
                    
                    optimizer.step()
                    
                except StopIteration as e:
                    print(f"An error occurred: {e}")   
                    # Catch StopIteration and continue to the next batch
                    pass
        except Exception as e:
            # Handle other exceptions if needed
            print(f"An error occurred: {e}")      

    def train_loop(self, generator, nb_epoch):
        
        self.to(self.device)

        data = generator.raw_data

        train_data = data.sample(frac=0.9,random_state=200)
        test_data = data.drop(train_data.index)

        test_data.reset_index(drop=True, inplace=True)
        train_data.reset_index(drop=True, inplace=True)

        test_generator = DataGenerator(test_data, filepath)
        test_loader = create_dataloader(test_generator)
        
        for epoch_number in range(nb_epoch):

            train_subdata = train_data.sample(frac=0.9,random_state=200)
            val_subdata = train_data.drop(train_subdata.index)
            
            train_subdata.reset_index(drop=True, inplace=True)
            val_subdata.reset_index(drop=True, inplace=True)
            
            train_subgenerator = DataGenerator(train_subdata, filepath)
            val_subgenerator = DataGenerator(val_subdata, filepath)

            train_loader = create_dataloader(train_subgenerator)
            val_loader = create_dataloader(val_subgenerator)

            print("")    
            print(f'{Fore.GREEN}EPOCH {epoch_number + 1}:{Style.RESET_ALL}')
            
            # Train for one epoch
            self.train_one_epoch(train_loader)

            # Validate on the validation subset
            print(f'{Fore.CYAN}Validation :{Style.RESET_ALL}')
            self.evaluate(val_loader)
            print(f'{Fore.YELLOW}Test :{Style.RESET_ALL}')
            self.evaluate(test_loader)

########################################################################

generator = DataGenerator(raw_data.iloc[2048:2048*3], filepath)
model = Model(27904, 16, 2, device)

dataloader = create_dataloader(generator)
model.evaluate(dataloader)

# print("Nombre de paramètres du model:", model.parameters_number())

# model.train_loop(generator, 5)

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Validating:   0%|          | 0/64 [00:00<?, ?it/s]


KeyError: 1460

In [10]:
import torch
from tqdm import tqdm
from colorama import Fore, Style
from data_processing import *
from transformers import Wav2Vec2Model
from transformers import CamembertModel
from icecream import ic

########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wave2vec_name = "facebook/wav2vec2-base-960h"
wave2vec_model = Wav2Vec2Model.from_pretrained(wave2vec_name)
wave2vec_model = wave2vec_model.to(device)

bert_name = 'camembert-base'
bert_model = CamembertModel.from_pretrained(bert_name)
bert_model = bert_model.to(device)

for param in wave2vec_model.parameters():
    param.requires_grad = False

for param in bert_model.parameters():
    param.requires_grad = False
    
########################################################################

class Model(torch.nn.Module):
    """
    Model using covolutional neural net architecture.
    """
    def __init__(self, input_dim, hidden_dim, output_dim, device):
        
        super(Model, self).__init__()

        self.device = device
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        # Define model components
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim)
        self.dropout = torch.nn.Dropout(p=0.1)
        self.linear2 = torch.nn.Linear(hidden_dim, output_dim)
        
###################
        
    def parameters_number(self):
        return(sum(p.numel() for p in self.parameters() if p.requires_grad))
        
    def forward(self, input_data):
        # Define the forward pass using the model components
        
        input_data['audio'] = input_data['audio'].to(self.device)
        input_data['text'] = input_data['text'].to(self.device)
        input_data['label'] = input_data['label'].to(self.device)

        channel0 = input_data['audio'][:,0,:]
        channel1 = input_data['audio'][:,1,:]
        
        wave2vec_output0 = wave2vec_model(channel0)
        wave2vec_output1 = wave2vec_model(channel1)
        
        wave2vec_output0 = wave2vec_output0.last_hidden_state
        wave2vec_output1 = wave2vec_output1.last_hidden_state
        
        wave2vec_output0 = torch.nn.functional.max_pool1d(wave2vec_output0, kernel_size=6)
        wave2vec_output1 = torch.nn.functional.max_pool1d(wave2vec_output1, kernel_size=6)
        
        bert_output = bert_model(input_data['text'])
        bert_output = bert_output.last_hidden_state
        
        wave2vec_output0 = torch.flatten(wave2vec_output0, start_dim=1)
        wave2vec_output1 = torch.flatten(wave2vec_output1, start_dim=1)
        bert_output = torch.flatten(bert_output, start_dim=1)

        # Concatenate or combine the outputs as needed
        combined_output = torch.cat((wave2vec_output0, wave2vec_output1, bert_output), dim=1)

        # Apply linear layers
        linear1_output = torch.relu(self.linear1(combined_output))
        final_output = self.linear2(self.dropout(linear1_output))

        return torch.softmax(final_output, dim=1)
    
    def evaluate(self, dataloader):
        self.eval()

        # Define the loss function
        loss_function = torch.nn.CrossEntropyLoss()

        total_loss = 0.0
        true_positive_1 = torch.tensor(0).to(self.device)
        false_positive_1 = torch.tensor(0).to(self.device)
        false_negative_1 = torch.tensor(0).to(self.device)
        
        true_positive_0 = torch.tensor(0).to(self.device)
        false_positive_0 = torch.tensor(0).to(self.device)
        false_negative_0 = torch.tensor(0).to(self.device)

        with torch.no_grad():  # Disable gradient computation during validation
            for _, batch in enumerate(tqdm(dataloader, desc="Validating")):
                
                output = self.forward(batch)
                labels = batch["label"].long()

                loss = loss_function(output, labels)
                total_loss += loss.item()

                _, predicted = torch.max(output, 1)
                
                true_positive_1 += torch.sum((predicted == labels) * (labels == 1))
                false_positive_1 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 1))
                false_negative_1 += torch.sum(((1-predicted) == labels) * (labels == 1))
                
                true_positive_0 += torch.sum((predicted == labels) * (labels == 0))
                false_positive_0 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 0))
                false_negative_0 += torch.sum(((1-predicted) == labels) * (labels == 0))
                
        precision_1 = true_positive_1 / max((true_positive_1 + false_positive_1), 1)
        recall_1 = true_positive_1 / max((true_positive_1 + false_negative_1), 1) 
        
        precision_0 = true_positive_0 / max((true_positive_0 + false_positive_0), 1)
        recall_0 = true_positive_0 / max((true_positive_0 + false_negative_0), 1) 
        
        f1_1 = 2 * (precision_1 * recall_1) / max((precision_1 + recall_1), 1)
        f1_0 = 2 * (precision_0 * recall_0) / max((precision_0 + recall_0), 1)
        
        print(f"Classe {Fore.RED}0{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_0 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_0 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_0 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Classe {Fore.RED}1{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_1 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_1 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_1 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Score : {(f1_0*0.18 + f1_1*(1-0.18))}")
                        
    def train_one_epoch(self, dataloader):
        
        self.train(True)
        
        optimizer = torch.optim.Adam(self.parameters())
        # loss_function = torch.nn.CrossEntropyLoss()
        
        try:
            for _, batch in enumerate(tqdm(dataloader, desc="Training")):
                
                try:
                
                    optimizer.zero_grad()
                    
                    output = self.forward(batch)
                    labels = batch["label"].long()
                    _, predicted = torch.max(output, 1)
                    
                    true_positive_0 = (output * (1 - labels).unsqueeze(1))[:, 0].sum()
                    false_positive_0 = ((1 - output) * labels.unsqueeze(1))[:, 0].sum()
                    false_negative_0 = ((1 - output) * (1 - labels).unsqueeze(1))[:, 0].sum()

                    true_positive_1 = (output * labels.unsqueeze(1))[:, 1].sum()
                    false_positive_1 = (output * (1 - labels).unsqueeze(1))[:, 1].sum()
                    false_negative_1 = ((1 - output) * labels.unsqueeze(1))[:, 1].sum()

                    epsilon = torch.tensor([1e-7]).to(self.device) # 1e-7
                    
                    precision_0 = true_positive_0 / (true_positive_0 + false_positive_0 + epsilon)
                    recall_0 = true_positive_0 / (true_positive_0 + false_negative_0 + epsilon)
                    f1_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0 + epsilon)

                    precision_1 = true_positive_1 / (true_positive_1 + false_positive_1 + epsilon)
                    recall_1 = true_positive_1 / (true_positive_1 + false_negative_1 + epsilon)
                    f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1 + epsilon)
                
                    loss = -torch.log((f1_0*0.18 + f1_1*(1-0.18)) + epsilon)
                    
                    # ic(output, labels)
                    # ic(true_positive_0, false_positive_0, false_negative_0, true_positive_1, false_positive_1, false_negative_1)
                    # ic(precision_0, recall_0, f1_0, precision_1, recall_1, f1_1)
                    # print("Successfully trained a batch")
                    
                    ic(loss)
                    loss.backward()
                    
                    # labels = batch["label"].long()
                    # loss = loss_function(output, labels)
                    # loss.backward()
                    
                    optimizer.step()
                    
                except StopIteration as e:
                    print(f"An error occurred: {e}")   
                    # Catch StopIteration and continue to the next batch
                    pass
        except Exception as e:
            # Handle other exceptions if needed
            print(f"An error occurred: {e}")      

    def train_loop(self, generator, nb_epoch):
        
        self.to(self.device)

        data = generator.raw_data

        train_data = data.sample(frac=0.9,random_state=200)
        test_data = data.drop(train_data.index)

        test_data.reset_index(drop=True, inplace=True)
        train_data.reset_index(drop=True, inplace=True)

        test_generator = DataGenerator(test_data, filepath)
        test_loader = create_dataloader(test_generator)
        
        for epoch_number in range(nb_epoch):

            train_subdata = train_data.sample(frac=0.9,random_state=200)
            val_subdata = train_data.drop(train_subdata.index)
            
            train_subdata.reset_index(drop=True, inplace=True)
            val_subdata.reset_index(drop=True, inplace=True)
            
            train_subgenerator = DataGenerator(train_subdata, filepath)
            val_subgenerator = DataGenerator(val_subdata, filepath)

            train_loader = create_dataloader(train_subgenerator)
            val_loader = create_dataloader(val_subgenerator)

            print("")    
            print(f'{Fore.GREEN}EPOCH {epoch_number + 1}:{Style.RESET_ALL}')
            
            # Train for one epoch
            self.train_one_epoch(train_loader)

            # Validate on the validation subset
            print(f'{Fore.CYAN}Validation :{Style.RESET_ALL}')
            self.evaluate(val_loader)
            print(f'{Fore.YELLOW}Test :{Style.RESET_ALL}')
            self.evaluate(test_loader)

########################################################################

generator = DataGenerator(raw_data.iloc[:64], filepath)
model = Model(27904, 16, 2, device)

dataloader = create_dataloader(generator)
model.evaluate(dataloader)

# print("Nombre de paramètres du model:", model.parameters_number())

# model.train_loop(generator, 5)

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Validating:   0%|          | 0/1 [00:27<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [11]:
import torch
from tqdm import tqdm
from colorama import Fore, Style
from data_processing import *
from transformers import Wav2Vec2Model
from transformers import CamembertModel
from icecream import ic

########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wave2vec_name = "facebook/wav2vec2-base-960h"
wave2vec_model = Wav2Vec2Model.from_pretrained(wave2vec_name)
wave2vec_model = wave2vec_model.to(device)

bert_name = 'camembert-base'
bert_model = CamembertModel.from_pretrained(bert_name)
bert_model = bert_model.to(device)

for param in wave2vec_model.parameters():
    param.requires_grad = False

for param in bert_model.parameters():
    param.requires_grad = False
    
########################################################################

class Model(torch.nn.Module):
    """
    Model using covolutional neural net architecture.
    """
    def __init__(self, input_dim, hidden_dim, output_dim, device):
        
        super(Model, self).__init__()

        self.device = device
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        # Define model components
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim)
        self.dropout = torch.nn.Dropout(p=0.1)
        self.linear2 = torch.nn.Linear(hidden_dim, output_dim)
        
###################
        
    def parameters_number(self):
        return(sum(p.numel() for p in self.parameters() if p.requires_grad))
        
    def forward(self, input_data):
        # Define the forward pass using the model components
        
        input_data['audio'] = input_data['audio'].to(self.device)
        input_data['text'] = input_data['text'].to(self.device)
        input_data['label'] = input_data['label'].to(self.device)

        channel0 = input_data['audio'][:,0,:]
        channel1 = input_data['audio'][:,1,:]
        
        wave2vec_output0 = wave2vec_model(channel0)
        wave2vec_output1 = wave2vec_model(channel1)
        
        wave2vec_output0 = wave2vec_output0.last_hidden_state
        wave2vec_output1 = wave2vec_output1.last_hidden_state
        
        wave2vec_output0 = torch.nn.functional.max_pool1d(wave2vec_output0, kernel_size=6)
        wave2vec_output1 = torch.nn.functional.max_pool1d(wave2vec_output1, kernel_size=6)
        
        bert_output = bert_model(input_data['text'])
        bert_output = bert_output.last_hidden_state
        
        wave2vec_output0 = torch.flatten(wave2vec_output0, start_dim=1)
        wave2vec_output1 = torch.flatten(wave2vec_output1, start_dim=1)
        bert_output = torch.flatten(bert_output, start_dim=1)

        # Concatenate or combine the outputs as needed
        combined_output = torch.cat((wave2vec_output0, wave2vec_output1, bert_output), dim=1)

        # Apply linear layers
        linear1_output = torch.relu(self.linear1(combined_output))
        final_output = self.linear2(self.dropout(linear1_output))

        return torch.softmax(final_output, dim=1)
    
    def evaluate(self, dataloader):
        
        self.to(self.device)
        
        self.eval()

        # Define the loss function
        loss_function = torch.nn.CrossEntropyLoss()

        total_loss = 0.0
        true_positive_1 = torch.tensor(0).to(self.device)
        false_positive_1 = torch.tensor(0).to(self.device)
        false_negative_1 = torch.tensor(0).to(self.device)
        
        true_positive_0 = torch.tensor(0).to(self.device)
        false_positive_0 = torch.tensor(0).to(self.device)
        false_negative_0 = torch.tensor(0).to(self.device)

        with torch.no_grad():  # Disable gradient computation during validation
            for _, batch in enumerate(tqdm(dataloader, desc="Validating")):
                
                output = self.forward(batch)
                labels = batch["label"].long()

                loss = loss_function(output, labels)
                total_loss += loss.item()

                _, predicted = torch.max(output, 1)
                
                true_positive_1 += torch.sum((predicted == labels) * (labels == 1))
                false_positive_1 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 1))
                false_negative_1 += torch.sum(((1-predicted) == labels) * (labels == 1))
                
                true_positive_0 += torch.sum((predicted == labels) * (labels == 0))
                false_positive_0 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 0))
                false_negative_0 += torch.sum(((1-predicted) == labels) * (labels == 0))
                
        precision_1 = true_positive_1 / max((true_positive_1 + false_positive_1), 1)
        recall_1 = true_positive_1 / max((true_positive_1 + false_negative_1), 1) 
        
        precision_0 = true_positive_0 / max((true_positive_0 + false_positive_0), 1)
        recall_0 = true_positive_0 / max((true_positive_0 + false_negative_0), 1) 
        
        f1_1 = 2 * (precision_1 * recall_1) / max((precision_1 + recall_1), 1)
        f1_0 = 2 * (precision_0 * recall_0) / max((precision_0 + recall_0), 1)
        
        print(f"Classe {Fore.RED}0{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_0 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_0 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_0 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Classe {Fore.RED}1{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_1 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_1 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_1 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Score : {(f1_0*0.18 + f1_1*(1-0.18))}")
                        
    def train_one_epoch(self, dataloader):
        
        self.train(True)
        
        optimizer = torch.optim.Adam(self.parameters())
        # loss_function = torch.nn.CrossEntropyLoss()
        
        try:
            for _, batch in enumerate(tqdm(dataloader, desc="Training")):
                
                try:
                
                    optimizer.zero_grad()
                    
                    output = self.forward(batch)
                    labels = batch["label"].long()
                    _, predicted = torch.max(output, 1)
                    
                    true_positive_0 = (output * (1 - labels).unsqueeze(1))[:, 0].sum()
                    false_positive_0 = ((1 - output) * labels.unsqueeze(1))[:, 0].sum()
                    false_negative_0 = ((1 - output) * (1 - labels).unsqueeze(1))[:, 0].sum()

                    true_positive_1 = (output * labels.unsqueeze(1))[:, 1].sum()
                    false_positive_1 = (output * (1 - labels).unsqueeze(1))[:, 1].sum()
                    false_negative_1 = ((1 - output) * labels.unsqueeze(1))[:, 1].sum()

                    epsilon = torch.tensor([1e-7]).to(self.device) # 1e-7
                    
                    precision_0 = true_positive_0 / (true_positive_0 + false_positive_0 + epsilon)
                    recall_0 = true_positive_0 / (true_positive_0 + false_negative_0 + epsilon)
                    f1_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0 + epsilon)

                    precision_1 = true_positive_1 / (true_positive_1 + false_positive_1 + epsilon)
                    recall_1 = true_positive_1 / (true_positive_1 + false_negative_1 + epsilon)
                    f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1 + epsilon)
                
                    loss = -torch.log((f1_0*0.18 + f1_1*(1-0.18)) + epsilon)
                    
                    # ic(output, labels)
                    # ic(true_positive_0, false_positive_0, false_negative_0, true_positive_1, false_positive_1, false_negative_1)
                    # ic(precision_0, recall_0, f1_0, precision_1, recall_1, f1_1)
                    # print("Successfully trained a batch")
                    
                    ic(loss)
                    loss.backward()
                    
                    # labels = batch["label"].long()
                    # loss = loss_function(output, labels)
                    # loss.backward()
                    
                    optimizer.step()
                    
                except StopIteration as e:
                    print(f"An error occurred: {e}")   
                    # Catch StopIteration and continue to the next batch
                    pass
        except Exception as e:
            # Handle other exceptions if needed
            print(f"An error occurred: {e}")      

    def train_loop(self, generator, nb_epoch):
        
        self.to(self.device)

        data = generator.raw_data

        train_data = data.sample(frac=0.9,random_state=200)
        test_data = data.drop(train_data.index)

        test_data.reset_index(drop=True, inplace=True)
        train_data.reset_index(drop=True, inplace=True)

        test_generator = DataGenerator(test_data, filepath)
        test_loader = create_dataloader(test_generator)
        
        for epoch_number in range(nb_epoch):

            train_subdata = train_data.sample(frac=0.9,random_state=200)
            val_subdata = train_data.drop(train_subdata.index)
            
            train_subdata.reset_index(drop=True, inplace=True)
            val_subdata.reset_index(drop=True, inplace=True)
            
            train_subgenerator = DataGenerator(train_subdata, filepath)
            val_subgenerator = DataGenerator(val_subdata, filepath)

            train_loader = create_dataloader(train_subgenerator)
            val_loader = create_dataloader(val_subgenerator)

            print("")    
            print(f'{Fore.GREEN}EPOCH {epoch_number + 1}:{Style.RESET_ALL}')
            
            # Train for one epoch
            self.train_one_epoch(train_loader)

            # Validate on the validation subset
            print(f'{Fore.CYAN}Validation :{Style.RESET_ALL}')
            self.evaluate(val_loader)
            print(f'{Fore.YELLOW}Test :{Style.RESET_ALL}')
            self.evaluate(test_loader)

########################################################################

generator = DataGenerator(raw_data.iloc[:64], filepath)
model = Model(27904, 16, 2, device)

dataloader = create_dataloader(generator)
model.evaluate(dataloader)

# print("Nombre de paramètres du model:", model.parameters_number())

# model.train_loop(generator, 5)

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Validating: 100%|██████████| 1/1 [00:23<00:00, 23.04s/it]

Classe 0 | Precision: 0.00%, Recall: 0.00%, F1 Score: 0.00%
Classe 1 | Precision: 15.62%, Recall: 100.00%, F1 Score: 27.03%
Score : 0.22162160277366638


In [12]:
import torch
from tqdm import tqdm
from colorama import Fore, Style
from data_processing import *
from transformers import Wav2Vec2Model
from transformers import CamembertModel
from icecream import ic

########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wave2vec_name = "facebook/wav2vec2-base-960h"
wave2vec_model = Wav2Vec2Model.from_pretrained(wave2vec_name)
wave2vec_model = wave2vec_model.to(device)

bert_name = 'camembert-base'
bert_model = CamembertModel.from_pretrained(bert_name)
bert_model = bert_model.to(device)

for param in wave2vec_model.parameters():
    param.requires_grad = False

for param in bert_model.parameters():
    param.requires_grad = False
    
########################################################################

class Model(torch.nn.Module):
    """
    Model using covolutional neural net architecture.
    """
    def __init__(self, input_dim, hidden_dim, output_dim, device):
        
        super(Model, self).__init__()

        self.device = device
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        # Define model components
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim)
        self.dropout = torch.nn.Dropout(p=0.1)
        self.linear2 = torch.nn.Linear(hidden_dim, output_dim)
        
###################
        
    def parameters_number(self):
        return(sum(p.numel() for p in self.parameters() if p.requires_grad))
        
    def forward(self, input_data):
        # Define the forward pass using the model components
        
        input_data['audio'] = input_data['audio'].to(self.device)
        input_data['text'] = input_data['text'].to(self.device)
        input_data['label'] = input_data['label'].to(self.device)

        channel0 = input_data['audio'][:,0,:]
        channel1 = input_data['audio'][:,1,:]
        
        wave2vec_output0 = wave2vec_model(channel0)
        wave2vec_output1 = wave2vec_model(channel1)
        
        wave2vec_output0 = wave2vec_output0.last_hidden_state
        wave2vec_output1 = wave2vec_output1.last_hidden_state
        
        wave2vec_output0 = torch.nn.functional.max_pool1d(wave2vec_output0, kernel_size=6)
        wave2vec_output1 = torch.nn.functional.max_pool1d(wave2vec_output1, kernel_size=6)
        
        bert_output = bert_model(input_data['text'])
        bert_output = bert_output.last_hidden_state
        
        wave2vec_output0 = torch.flatten(wave2vec_output0, start_dim=1)
        wave2vec_output1 = torch.flatten(wave2vec_output1, start_dim=1)
        bert_output = torch.flatten(bert_output, start_dim=1)

        # Concatenate or combine the outputs as needed
        combined_output = torch.cat((wave2vec_output0, wave2vec_output1, bert_output), dim=1)

        # Apply linear layers
        linear1_output = torch.relu(self.linear1(combined_output))
        final_output = self.linear2(self.dropout(linear1_output))

        return torch.softmax(final_output, dim=1)
    
    def evaluate(self, dataloader):
        
        self.to(self.device)
        
        self.eval()

        # Define the loss function
        loss_function = torch.nn.CrossEntropyLoss()

        total_loss = 0.0
        true_positive_1 = torch.tensor(0).to(self.device)
        false_positive_1 = torch.tensor(0).to(self.device)
        false_negative_1 = torch.tensor(0).to(self.device)
        
        true_positive_0 = torch.tensor(0).to(self.device)
        false_positive_0 = torch.tensor(0).to(self.device)
        false_negative_0 = torch.tensor(0).to(self.device)

        with torch.no_grad():  # Disable gradient computation during validation
            for _, batch in enumerate(tqdm(dataloader, desc="Validating")):
                
                output = self.forward(batch)
                labels = batch["label"].long()

                loss = loss_function(output, labels)
                total_loss += loss.item()

                _, predicted = torch.max(output, 1)
                
                true_positive_1 += torch.sum((predicted == labels) * (labels == 1))
                false_positive_1 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 1))
                false_negative_1 += torch.sum(((1-predicted) == labels) * (labels == 1))
                
                true_positive_0 += torch.sum((predicted == labels) * (labels == 0))
                false_positive_0 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 0))
                false_negative_0 += torch.sum(((1-predicted) == labels) * (labels == 0))
                
        precision_1 = true_positive_1 / max((true_positive_1 + false_positive_1), 1)
        recall_1 = true_positive_1 / max((true_positive_1 + false_negative_1), 1) 
        
        precision_0 = true_positive_0 / max((true_positive_0 + false_positive_0), 1)
        recall_0 = true_positive_0 / max((true_positive_0 + false_negative_0), 1) 
        
        f1_1 = 2 * (precision_1 * recall_1) / max((precision_1 + recall_1), 1)
        f1_0 = 2 * (precision_0 * recall_0) / max((precision_0 + recall_0), 1)
        
        print(f"Classe {Fore.RED}0{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_0 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_0 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_0 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Classe {Fore.RED}1{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_1 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_1 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_1 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Score : {(f1_0*0.18 + f1_1*(1-0.18))}")
                        
    def train_one_epoch(self, dataloader):
        
        self.train(True)
        
        optimizer = torch.optim.Adam(self.parameters())
        # loss_function = torch.nn.CrossEntropyLoss()
        
        try:
            for _, batch in enumerate(tqdm(dataloader, desc="Training")):
                
                try:
                
                    optimizer.zero_grad()
                    
                    output = self.forward(batch)
                    labels = batch["label"].long()
                    _, predicted = torch.max(output, 1)
                    
                    true_positive_0 = (output * (1 - labels).unsqueeze(1))[:, 0].sum()
                    false_positive_0 = ((1 - output) * labels.unsqueeze(1))[:, 0].sum()
                    false_negative_0 = ((1 - output) * (1 - labels).unsqueeze(1))[:, 0].sum()

                    true_positive_1 = (output * labels.unsqueeze(1))[:, 1].sum()
                    false_positive_1 = (output * (1 - labels).unsqueeze(1))[:, 1].sum()
                    false_negative_1 = ((1 - output) * labels.unsqueeze(1))[:, 1].sum()

                    epsilon = torch.tensor([1e-7]).to(self.device) # 1e-7
                    
                    precision_0 = true_positive_0 / (true_positive_0 + false_positive_0 + epsilon)
                    recall_0 = true_positive_0 / (true_positive_0 + false_negative_0 + epsilon)
                    f1_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0 + epsilon)

                    precision_1 = true_positive_1 / (true_positive_1 + false_positive_1 + epsilon)
                    recall_1 = true_positive_1 / (true_positive_1 + false_negative_1 + epsilon)
                    f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1 + epsilon)
                
                    loss = -torch.log((f1_0*0.18 + f1_1*(1-0.18)) + epsilon)
                    
                    # ic(output, labels)
                    # ic(true_positive_0, false_positive_0, false_negative_0, true_positive_1, false_positive_1, false_negative_1)
                    # ic(precision_0, recall_0, f1_0, precision_1, recall_1, f1_1)
                    # print("Successfully trained a batch")
                    
                    ic(loss)
                    loss.backward()
                    
                    # labels = batch["label"].long()
                    # loss = loss_function(output, labels)
                    # loss.backward()
                    
                    optimizer.step()
                    
                except StopIteration as e:
                    print(f"An error occurred: {e}")   
                    # Catch StopIteration and continue to the next batch
                    pass
        except Exception as e:
            # Handle other exceptions if needed
            print(f"An error occurred: {e}")      

    def train_loop(self, generator, nb_epoch):
        
        self.to(self.device)

        data = generator.raw_data

        train_data = data.sample(frac=0.9,random_state=200)
        test_data = data.drop(train_data.index)

        test_data.reset_index(drop=True, inplace=True)
        train_data.reset_index(drop=True, inplace=True)

        test_generator = DataGenerator(test_data, filepath)
        test_loader = create_dataloader(test_generator)
        
        for epoch_number in range(nb_epoch):

            train_subdata = train_data.sample(frac=0.9,random_state=200)
            val_subdata = train_data.drop(train_subdata.index)
            
            train_subdata.reset_index(drop=True, inplace=True)
            val_subdata.reset_index(drop=True, inplace=True)
            
            train_subgenerator = DataGenerator(train_subdata, filepath)
            val_subgenerator = DataGenerator(val_subdata, filepath)

            train_loader = create_dataloader(train_subgenerator)
            val_loader = create_dataloader(val_subgenerator)

            print("")    
            print(f'{Fore.GREEN}EPOCH {epoch_number + 1}:{Style.RESET_ALL}')
            
            # Train for one epoch
            self.train_one_epoch(train_loader)

            # Validate on the validation subset
            print(f'{Fore.CYAN}Validation :{Style.RESET_ALL}')
            self.evaluate(val_loader)
            print(f'{Fore.YELLOW}Test :{Style.RESET_ALL}')
            self.evaluate(test_loader)

########################################################################

generator = DataGenerator(raw_data.iloc[:64*4], filepath)
model = Model(27904, 16, 2, device)

dataloader = create_dataloader(generator)
model.evaluate(dataloader)

# print("Nombre de paramètres du model:", model.parameters_number())

# model.train_loop(generator, 5)

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Validating: 100%|██████████| 4/4 [01:49<00:00, 27.47s/it]

Classe 0 | Precision: 82.81%, Recall: 100.00%, F1 Score: 90.60%
Classe 1 | Precision: 0.00%, Recall: 0.00%, F1 Score: 0.00%
Score : 0.16307693719863892


In [13]:
model = torch.load("checkpoint1.pth")

In [14]:
import torch
from tqdm import tqdm
from colorama import Fore, Style
from data_processing import *
from transformers import Wav2Vec2Model
from transformers import CamembertModel
from icecream import ic

########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wave2vec_name = "facebook/wav2vec2-base-960h"
wave2vec_model = Wav2Vec2Model.from_pretrained(wave2vec_name)
wave2vec_model = wave2vec_model.to(device)

bert_name = 'camembert-base'
bert_model = CamembertModel.from_pretrained(bert_name)
bert_model = bert_model.to(device)

for param in wave2vec_model.parameters():
    param.requires_grad = False

for param in bert_model.parameters():
    param.requires_grad = False
    
########################################################################

class Model(torch.nn.Module):
    """
    Model using covolutional neural net architecture.
    """
    def __init__(self, input_dim, hidden_dim, output_dim, device):
        
        super(Model, self).__init__()

        self.device = device
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        # Define model components
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim)
        self.dropout = torch.nn.Dropout(p=0.1)
        self.linear2 = torch.nn.Linear(hidden_dim, output_dim)
        
###################
        
    def parameters_number(self):
        return(sum(p.numel() for p in self.parameters() if p.requires_grad))
        
    def forward(self, input_data):
        # Define the forward pass using the model components
        
        input_data['audio'] = input_data['audio'].to(self.device)
        input_data['text'] = input_data['text'].to(self.device)
        input_data['label'] = input_data['label'].to(self.device)

        channel0 = input_data['audio'][:,0,:]
        channel1 = input_data['audio'][:,1,:]
        
        wave2vec_output0 = wave2vec_model(channel0)
        wave2vec_output1 = wave2vec_model(channel1)
        
        wave2vec_output0 = wave2vec_output0.last_hidden_state
        wave2vec_output1 = wave2vec_output1.last_hidden_state
        
        wave2vec_output0 = torch.nn.functional.max_pool1d(wave2vec_output0, kernel_size=6)
        wave2vec_output1 = torch.nn.functional.max_pool1d(wave2vec_output1, kernel_size=6)
        
        bert_output = bert_model(input_data['text'])
        bert_output = bert_output.last_hidden_state
        
        wave2vec_output0 = torch.flatten(wave2vec_output0, start_dim=1)
        wave2vec_output1 = torch.flatten(wave2vec_output1, start_dim=1)
        bert_output = torch.flatten(bert_output, start_dim=1)

        # Concatenate or combine the outputs as needed
        combined_output = torch.cat((wave2vec_output0, wave2vec_output1, bert_output), dim=1)

        # Apply linear layers
        linear1_output = torch.relu(self.linear1(combined_output))
        final_output = self.linear2(self.dropout(linear1_output))

        return torch.softmax(final_output, dim=1)
    
    def evaluate(self, dataloader):
        
        self.to(self.device)
        
        self.eval()

        # Define the loss function
        loss_function = torch.nn.CrossEntropyLoss()

        total_loss = 0.0
        true_positive_1 = torch.tensor(0).to(self.device)
        false_positive_1 = torch.tensor(0).to(self.device)
        false_negative_1 = torch.tensor(0).to(self.device)
        
        true_positive_0 = torch.tensor(0).to(self.device)
        false_positive_0 = torch.tensor(0).to(self.device)
        false_negative_0 = torch.tensor(0).to(self.device)

        with torch.no_grad():  # Disable gradient computation during validation
            for _, batch in enumerate(tqdm(dataloader, desc="Validating")):
                
                output = self.forward(batch)
                labels = batch["label"].long()

                loss = loss_function(output, labels)
                total_loss += loss.item()

                _, predicted = torch.max(output, 1)
                
                true_positive_1 += torch.sum((predicted == labels) * (labels == 1))
                false_positive_1 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 1))
                false_negative_1 += torch.sum(((1-predicted) == labels) * (labels == 1))
                
                true_positive_0 += torch.sum((predicted == labels) * (labels == 0))
                false_positive_0 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 0))
                false_negative_0 += torch.sum(((1-predicted) == labels) * (labels == 0))
                
        precision_1 = true_positive_1 / max((true_positive_1 + false_positive_1), 1)
        recall_1 = true_positive_1 / max((true_positive_1 + false_negative_1), 1) 
        
        precision_0 = true_positive_0 / max((true_positive_0 + false_positive_0), 1)
        recall_0 = true_positive_0 / max((true_positive_0 + false_negative_0), 1) 
        
        f1_1 = 2 * (precision_1 * recall_1) / max((precision_1 + recall_1), 1)
        f1_0 = 2 * (precision_0 * recall_0) / max((precision_0 + recall_0), 1)
        
        print(f"Classe {Fore.RED}0{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_0 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_0 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_0 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Classe {Fore.RED}1{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_1 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_1 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_1 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Score : {(f1_0*0.18 + f1_1*(1-0.18))}")
                        
    def train_one_epoch(self, dataloader):
        
        self.train(True)
        
        optimizer = torch.optim.Adam(self.parameters())
        # loss_function = torch.nn.CrossEntropyLoss()
        
        try:
            for _, batch in enumerate(tqdm(dataloader, desc="Training")):
                
                try:
                
                    optimizer.zero_grad()
                    
                    output = self.forward(batch)
                    labels = batch["label"].long()
                    _, predicted = torch.max(output, 1)
                    
                    true_positive_0 = (output * (1 - labels).unsqueeze(1))[:, 0].sum()
                    false_positive_0 = ((1 - output) * labels.unsqueeze(1))[:, 0].sum()
                    false_negative_0 = ((1 - output) * (1 - labels).unsqueeze(1))[:, 0].sum()

                    true_positive_1 = (output * labels.unsqueeze(1))[:, 1].sum()
                    false_positive_1 = (output * (1 - labels).unsqueeze(1))[:, 1].sum()
                    false_negative_1 = ((1 - output) * labels.unsqueeze(1))[:, 1].sum()

                    epsilon = torch.tensor([1e-7]).to(self.device) # 1e-7
                    
                    precision_0 = true_positive_0 / (true_positive_0 + false_positive_0 + epsilon)
                    recall_0 = true_positive_0 / (true_positive_0 + false_negative_0 + epsilon)
                    f1_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0 + epsilon)

                    precision_1 = true_positive_1 / (true_positive_1 + false_positive_1 + epsilon)
                    recall_1 = true_positive_1 / (true_positive_1 + false_negative_1 + epsilon)
                    f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1 + epsilon)
                
                    loss = -torch.log((f1_0*0.18 + f1_1*(1-0.18)) + epsilon)
                    
                    # ic(output, labels)
                    # ic(true_positive_0, false_positive_0, false_negative_0, true_positive_1, false_positive_1, false_negative_1)
                    # ic(precision_0, recall_0, f1_0, precision_1, recall_1, f1_1)
                    # print("Successfully trained a batch")
                    
                    ic(loss)
                    loss.backward()
                    
                    # labels = batch["label"].long()
                    # loss = loss_function(output, labels)
                    # loss.backward()
                    
                    optimizer.step()
                    
                except StopIteration as e:
                    print(f"An error occurred: {e}")   
                    # Catch StopIteration and continue to the next batch
                    pass
        except Exception as e:
            # Handle other exceptions if needed
            print(f"An error occurred: {e}")      

    def train_loop(self, generator, nb_epoch):
        
        self.to(self.device)

        data = generator.raw_data

        train_data = data.sample(frac=0.9,random_state=200)
        test_data = data.drop(train_data.index)

        test_data.reset_index(drop=True, inplace=True)
        train_data.reset_index(drop=True, inplace=True)

        test_generator = DataGenerator(test_data, filepath)
        test_loader = create_dataloader(test_generator)
        
        for epoch_number in range(nb_epoch):

            train_subdata = train_data.sample(frac=0.9,random_state=200)
            val_subdata = train_data.drop(train_subdata.index)
            
            train_subdata.reset_index(drop=True, inplace=True)
            val_subdata.reset_index(drop=True, inplace=True)
            
            train_subgenerator = DataGenerator(train_subdata, filepath)
            val_subgenerator = DataGenerator(val_subdata, filepath)

            train_loader = create_dataloader(train_subgenerator)
            val_loader = create_dataloader(val_subgenerator)

            print("")    
            print(f'{Fore.GREEN}EPOCH {epoch_number + 1}:{Style.RESET_ALL}')
            
            # Train for one epoch
            self.train_one_epoch(train_loader)

            # Validate on the validation subset
            print(f'{Fore.CYAN}Validation :{Style.RESET_ALL}')
            self.evaluate(val_loader)
            print(f'{Fore.YELLOW}Test :{Style.RESET_ALL}')
            self.evaluate(test_loader)

########################################################################

generator = DataGenerator(raw_data.iloc[4096:8192], filepath)

# model = Model(27904, 16, 2, device)
# dataloader = create_dataloader(generator)
# model.evaluate(dataloader)

# print("Nombre de paramètres du model:", model.parameters_number())

# model.train_loop(generator, 5)

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model.train_loop(generator, 5)


EPOCH 1:


Training:   0%|          | 0/51 [00:10<?, ?it/s]


An error occurred: Unable to create tensor, you should probably activate padding with 'padding=True' to have batched tensors with the same length.
Validation :


Validating:   0%|          | 0/5 [00:00<?, ?it/s]

In [1]:
model = torch.load("checkpoint1")

NameError: name 'torch' is not defined

In [2]:
import torch
from tqdm import tqdm
from colorama import Fore, Style
from data_processing import *
from transformers import Wav2Vec2Model
from transformers import CamembertModel
from icecream import ic

########################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wave2vec_name = "facebook/wav2vec2-base-960h"
wave2vec_model = Wav2Vec2Model.from_pretrained(wave2vec_name)
wave2vec_model = wave2vec_model.to(device)

bert_name = 'camembert-base'
bert_model = CamembertModel.from_pretrained(bert_name)
bert_model = bert_model.to(device)

for param in wave2vec_model.parameters():
    param.requires_grad = False

for param in bert_model.parameters():
    param.requires_grad = False
    
########################################################################

class Model(torch.nn.Module):
    """
    Model using covolutional neural net architecture.
    """
    def __init__(self, input_dim, hidden_dim, output_dim, device):
        
        super(Model, self).__init__()

        self.device = device
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        # Define model components
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim)
        self.dropout = torch.nn.Dropout(p=0.1)
        self.linear2 = torch.nn.Linear(hidden_dim, output_dim)
        
###################
        
    def parameters_number(self):
        return(sum(p.numel() for p in self.parameters() if p.requires_grad))
        
    def forward(self, input_data):
        # Define the forward pass using the model components
        
        input_data['audio'] = input_data['audio'].to(self.device)
        input_data['text'] = input_data['text'].to(self.device)
        input_data['label'] = input_data['label'].to(self.device)

        channel0 = input_data['audio'][:,0,:]
        channel1 = input_data['audio'][:,1,:]
        
        wave2vec_output0 = wave2vec_model(channel0)
        wave2vec_output1 = wave2vec_model(channel1)
        
        wave2vec_output0 = wave2vec_output0.last_hidden_state
        wave2vec_output1 = wave2vec_output1.last_hidden_state
        
        wave2vec_output0 = torch.nn.functional.max_pool1d(wave2vec_output0, kernel_size=6)
        wave2vec_output1 = torch.nn.functional.max_pool1d(wave2vec_output1, kernel_size=6)
        
        bert_output = bert_model(input_data['text'])
        bert_output = bert_output.last_hidden_state
        
        wave2vec_output0 = torch.flatten(wave2vec_output0, start_dim=1)
        wave2vec_output1 = torch.flatten(wave2vec_output1, start_dim=1)
        bert_output = torch.flatten(bert_output, start_dim=1)

        # Concatenate or combine the outputs as needed
        combined_output = torch.cat((wave2vec_output0, wave2vec_output1, bert_output), dim=1)

        # Apply linear layers
        linear1_output = torch.relu(self.linear1(combined_output))
        final_output = self.linear2(self.dropout(linear1_output))

        return torch.softmax(final_output, dim=1)
    
    def evaluate(self, dataloader):
        
        self.to(self.device)
        
        self.eval()

        # Define the loss function
        loss_function = torch.nn.CrossEntropyLoss()

        total_loss = 0.0
        true_positive_1 = torch.tensor(0).to(self.device)
        false_positive_1 = torch.tensor(0).to(self.device)
        false_negative_1 = torch.tensor(0).to(self.device)
        
        true_positive_0 = torch.tensor(0).to(self.device)
        false_positive_0 = torch.tensor(0).to(self.device)
        false_negative_0 = torch.tensor(0).to(self.device)

        with torch.no_grad():  # Disable gradient computation during validation
            for _, batch in enumerate(tqdm(dataloader, desc="Validating")):
                
                output = self.forward(batch)
                labels = batch["label"].long()

                loss = loss_function(output, labels)
                total_loss += loss.item()

                _, predicted = torch.max(output, 1)
                
                true_positive_1 += torch.sum((predicted == labels) * (labels == 1))
                false_positive_1 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 1))
                false_negative_1 += torch.sum(((1-predicted) == labels) * (labels == 1))
                
                true_positive_0 += torch.sum((predicted == labels) * (labels == 0))
                false_positive_0 += torch.sum((predicted == (1 - labels)) * ((1 - labels) == 0))
                false_negative_0 += torch.sum(((1-predicted) == labels) * (labels == 0))
                
        precision_1 = true_positive_1 / max((true_positive_1 + false_positive_1), 1)
        recall_1 = true_positive_1 / max((true_positive_1 + false_negative_1), 1) 
        
        precision_0 = true_positive_0 / max((true_positive_0 + false_positive_0), 1)
        recall_0 = true_positive_0 / max((true_positive_0 + false_negative_0), 1) 
        
        f1_1 = 2 * (precision_1 * recall_1) / max((precision_1 + recall_1), 1)
        f1_0 = 2 * (precision_0 * recall_0) / max((precision_0 + recall_0), 1)
        
        print(f"Classe {Fore.RED}0{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_0 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_0 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_0 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Classe {Fore.RED}1{Style.RESET_ALL} | Precision: {Fore.GREEN}{precision_1 * 100:.2f}%{Style.RESET_ALL}, Recall: {Fore.GREEN}{recall_1 * 100:.2f}%{Style.RESET_ALL}, F1 Score: {Fore.GREEN}{f1_1 * 100:.2f}%{Style.RESET_ALL}")
        print(f"Score : {(f1_0*0.18 + f1_1*(1-0.18))}")
                        
    def train_one_epoch(self, dataloader):
        
        self.train(True)
        
        optimizer = torch.optim.Adam(self.parameters())
        # loss_function = torch.nn.CrossEntropyLoss()
        
        try:
            for _, batch in enumerate(tqdm(dataloader, desc="Training")):
                
                try:
                
                    optimizer.zero_grad()
                    
                    output = self.forward(batch)
                    labels = batch["label"].long()
                    _, predicted = torch.max(output, 1)
                    
                    true_positive_0 = (output * (1 - labels).unsqueeze(1))[:, 0].sum()
                    false_positive_0 = ((1 - output) * labels.unsqueeze(1))[:, 0].sum()
                    false_negative_0 = ((1 - output) * (1 - labels).unsqueeze(1))[:, 0].sum()

                    true_positive_1 = (output * labels.unsqueeze(1))[:, 1].sum()
                    false_positive_1 = (output * (1 - labels).unsqueeze(1))[:, 1].sum()
                    false_negative_1 = ((1 - output) * labels.unsqueeze(1))[:, 1].sum()

                    epsilon = torch.tensor([1e-7]).to(self.device) # 1e-7
                    
                    precision_0 = true_positive_0 / (true_positive_0 + false_positive_0 + epsilon)
                    recall_0 = true_positive_0 / (true_positive_0 + false_negative_0 + epsilon)
                    f1_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0 + epsilon)

                    precision_1 = true_positive_1 / (true_positive_1 + false_positive_1 + epsilon)
                    recall_1 = true_positive_1 / (true_positive_1 + false_negative_1 + epsilon)
                    f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1 + epsilon)
                
                    loss = -torch.log((f1_0*0.18 + f1_1*(1-0.18)) + epsilon)
                    
                    # ic(output, labels)
                    # ic(true_positive_0, false_positive_0, false_negative_0, true_positive_1, false_positive_1, false_negative_1)
                    # ic(precision_0, recall_0, f1_0, precision_1, recall_1, f1_1)
                    # print("Successfully trained a batch")
                    
                    ic(loss)
                    loss.backward()
                    
                    # labels = batch["label"].long()
                    # loss = loss_function(output, labels)
                    # loss.backward()
                    
                    optimizer.step()
                    
                except StopIteration as e:
                    print(f"An error occurred: {e}")   
                    # Catch StopIteration and continue to the next batch
                    pass
        except Exception as e:
            # Handle other exceptions if needed
            print(f"An error occurred: {e}")      

    def train_loop(self, generator, nb_epoch):
        
        self.to(self.device)

        data = generator.raw_data

        train_data = data.sample(frac=0.9,random_state=200)
        test_data = data.drop(train_data.index)

        test_data.reset_index(drop=True, inplace=True)
        train_data.reset_index(drop=True, inplace=True)

        test_generator = DataGenerator(test_data, filepath)
        test_loader = create_dataloader(test_generator)
        
        for epoch_number in range(nb_epoch):

            train_subdata = train_data.sample(frac=0.9,random_state=200)
            val_subdata = train_data.drop(train_subdata.index)
            
            train_subdata.reset_index(drop=True, inplace=True)
            val_subdata.reset_index(drop=True, inplace=True)
            
            train_subgenerator = DataGenerator(train_subdata, filepath)
            val_subgenerator = DataGenerator(val_subdata, filepath)

            train_loader = create_dataloader(train_subgenerator)
            val_loader = create_dataloader(val_subgenerator)

            print("")    
            print(f'{Fore.GREEN}EPOCH {epoch_number + 1}:{Style.RESET_ALL}')
            
            # Train for one epoch
            self.train_one_epoch(train_loader)

            # Validate on the validation subset
            print(f'{Fore.CYAN}Validation :{Style.RESET_ALL}')
            self.evaluate(val_loader)
            print(f'{Fore.YELLOW}Test :{Style.RESET_ALL}')
            self.evaluate(test_loader)

########################################################################

generator = DataGenerator(raw_data.iloc[8192:8192+4096], filepath)

# model = Model(27904, 16, 2, device)
# dataloader = create_dataloader(generator)
# model.evaluate(dataloader)

# print("Nombre de paramètres du model:", model.parameters_number())

# model.train_loop(generator, 5)

(16400, 12)
is_main_speaker      0.739207
turn_at_start        0.184756
turn_after           0.184756
turn_start_word     10.250372
yield_at_end         0.188415
request_at_start     0.198537
dtype: float64


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
model = torch.load("checkpoint1")

FileNotFoundError: [Errno 2] No such file or directory: 'checkpoint1'

In [4]:
model = torch.load("checkpoint1.pth")

In [5]:
model.train_loop(generator, 5)


EPOCH 1:


Training:   0%|          | 0/51 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
ic| loss: tensor([1.0002], device='cuda:0', grad_fn=<NegBackward0>)
Training:  55%|█████▍    | 28/51 [20:11<16:35, 43.27s/it]


An error occurred: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).
Validation :


Validating: 100%|██████████| 5/5 [03:30<00:00, 42.04s/it]


Classe 0 | Precision: 81.47%, Recall: 84.40%, F1 Score: 82.91%
Classe 1 | Precision: 36.07%, Recall: 31.43%, F1 Score: 22.67%
Score : 0.33512604236602783
Test :


...

Classe 0 | Precision: 82.95%, Recall: 72.87%, F1 Score: 77.59%
Classe 1 | Precision: 34.95%, Recall: 49.32%, F1 Score: 34.47%
Score : 0.4223310947418213
Test :


Validating: 100%|██████████| 6/6 [03:52<00:00, 38.75s/it]


Classe 0 | Precision: 89.52%, Recall: 68.73%, F1 Score: 77.76%
Classe 1 | Precision: 25.74%, Recall: 57.38%, F1 Score: 29.53%
Score : 0.38212987780570984

EPOCH 3:


Training:  43%|████▎     | 22/51 [16:02<21:08, 43.75s/it]


KeyboardInterrupt: 

In [6]:
torch.save(model, "checkpoint2.pth")